In [1]:
import sys
sys.path.insert(0, '..')

In [5]:
import os
import datetime
import pandas as pd
import numpy as np
from tinkoff.invest import Client
from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.schemas import InstrumentStatus, InstrumentExchangeType
from src.tinkoff_convert import _dataclass_to_dict, _dataclasses_to_dict
from src.tinkoff_api import get_prices_sync, get_dividends_async, get_raw_positions, get_raw_portfolio
from robot import PortfolioManager
from execution import MockManager
from enum import IntEnum

In [3]:
TOKEN = os.environ["TINVEST_READ"]

In [4]:
with Client(TOKEN) as client:
    # shares_arr = client.instruments.shares(instrument_status = InstrumentStatus.INSTRUMENT_STATUS_ALL)
    # assets_resp = client.instruments.get_assets()
    # asset_resp = client.instruments.get_asset_by(id = '94f18cc6-9e01-4db6-9d73-8c3ba6ab9655')
    accs = client.users.get_accounts()
    portfolio = get_raw_portfolio(account_id='2169054877', token=TOKEN)
    positions = get_raw_positions(account_id='2169054877', token=TOKEN)

In [12]:
portfolio_df = pd.DataFrame(_dataclasses_to_dict(portfolio.positions))

In [6]:
positions_df = pd.DataFrame(_dataclasses_to_dict(positions.securities))
positions_df

,figi,blocked,balance,position_uid,instrument_uid,exchange_blocked,instrument_type
0,BBG00N8J90K7,0,6,a0e2bd0d-2782-4836-b547-0f591d98ba93,c355adac-9536-4aed-a09f-15bc4e99d4b5,False,bond
1,BBG013RTHWK8,0,200,8186b92b-62b2-42ad-96c2-6158566a1e7d,49ddaef5-13e8-48fb-b16f-7f940f475151,False,etf
2,TCS00A103D37,0,173,461af849-8476-4900-bc59-60f1af729eba,dfbb0ff1-67cc-410e-bfe8-6a62d845b50c,False,bond
3,BBG00B9PJ7V0,0,4,8a37365a-8fe3-4fcb-93e0-924ef984e3cc,a29f7dc2-1eed-4db8-a14d-bf1f5d881ab4,False,bond
4,BBG00XH4W3N3,0,13,2c354d2c-98d0-4705-8370-92e604e31ece,2dd3b003-aca2-4920-89ce-8d827c637372,False,bond
5,BBG004730N88,0,100,41eb2102-5333-4713-bf15-72b204c4bf7b,e6123145-9665-43e0-8413-cd61b8aa9b13,False,share
6,BBG011616RM7,0,10,80edfa26-c0a8-41ea-9d9c-216fd34c6f22,ea070157-241a-4b42-9d85-a17f01fdb936,False,bond
7,BBG00Y90V3M0,0,1,8fd0ca97-edd8-4ed2-b3bc-d443dea8789b,32decb51-bd31-461e-8f87-fa7ecff82952,False,bond
8,BBG011MDBQV9,0,7,1acc547f-f8b6-486d-a0ca-f2bd9d092d09,44d709e2-a362-417d-92e3-36996f45b747,False,bond
9,BBG333333333,0,500,29888afc-181d-45ed-a00b-8fe55a9cec9b,9654c2dd-6993-427e-80fa-04e80a1cf4da,False,etf


In [15]:
pd.DataFrame(portfolio_dict).to_csv('portfolio.csv', index = False)
positions_df.to_csv('positions.csv', index = False)

In [4]:
portfolio_df = pd.read_csv('../portfolio.csv')
portfolio_df['lot'] = portfolio_df.eval('quantity/quantity_lots').astype('int')
positions_df = pd.read_csv('positions.csv')

In [6]:
order_manager = MockManager('../data/operations.duckdb')
manager = PortfolioManager(order_manager, path_db='../data/main.duckdb')

In [7]:
target_weights = manager.allocate(datetime.datetime.now(), imoex_content='../data/imoex_content.xlsx')

In [8]:
# res = manager.prepare_positions(positions_df.rename(columns = {'balance': 'quantity'}), target_weights)
res = manager.prepare_positions(portfolio_df[['figi', 'instrument_type', 'quantity', 'current_price', 'lot']],
                                target_weights)
res['instrument_type'] = res['instrument_type'].fillna('share')

In [17]:
positions = res.reset_index().to_dict('records')

In [20]:
position

{'index': 'BBG004730N88',
 'instrument_type': 'share',
 'quantity': 100.0,
 'current_price': 249.23,
 'lot': 10.0,
 'ticker': 'SBER',
 'market_value': 24923.0,
 'weight': 0.3394389794577322,
 'target_weight': 0.02245709457888359,
 'delta_weight': -0.3169818848788486,
 'target_market_value': 1648.8918540930586,
 'delta_market_value': -23274.108145906943,
 'delta_quantity_for_order': -9}

In [23]:
import duckdb
with duckdb.connect("../data/operations.duckdb") as conn:
    order_df = conn.sql("SELECT * FROM order_history").df()
order_df

,figi,quantity,price,direction,account_id,order_type,order_id,record_dtime
0,BBG004730N88,9,249.229996,2,test,1,b76e09bc-d145-4095-8739-d6b796fceb85,2024-11-18 23:21:20.724629
1,TCS00A103D37,173,96.192688,2,test,1,0044cf26-808e-47fc-8c77-c3b9b654856c,2024-11-18 23:21:20.780242
2,BBG00XH4W3N3,13,567.000000,2,test,1,1408ad95-828f-4acb-a9a7-ccd489dbf51f,2024-11-18 23:21:20.811123
3,BBG011616RM7,10,713.200012,2,test,1,a1c1422a-95b7-4939-837d-b184c1359c5b,2024-11-18 23:21:20.840825
4,BBG00N8J90K7,6,821.099976,2,test,1,130f9902-0717-4b1a-b24f-f86dcefa979f,2024-11-18 23:21:20.870883
5,BBG011MDBQV9,7,701.299988,2,test,1,66d76a8b-a05f-47db-b7df-a3cf5c8025d5,2024-11-18 23:21:20.898407
6,BBG333333333,500,5.910000,2,test,1,f2ebcab4-2151-413e-9cbc-fce2eb3b7e36,2024-11-18 23:21:20.923236
7,BBG00B9PJ7V0,4,669.989990,2,test,1,77b28a3c-3d6a-44a5-b549-119458485338,2024-11-18 23:21:20.948329
8,BBG013RTHWK8,200,8.112000,2,test,1,c2cc4002-0b03-4baf-8241-c34f8cfdc139,2024-11-18 23:21:20.973000
9,BBG00Y90V3M0,1,263.700012,2,test,1,a64a240b-d48f-4628-a3c8-0c53f843d1c2,2024-11-18 23:21:20.998474


In [15]:
res.groupby('instrument_type').agg({'delta_quantity_for_order': [
    ('long',  lambda x: (x > 0).sum()), 
    ('short',  lambda x: (x < 0).sum()),
    ('hold', lambda x: (x == 0).sum())]})

delta_quantity_for_order           
                                    long short hold
instrument_type                                    
bond                                   0     7    0
etf                                    0     2    0
share                                 10     1    0

## src lib

In [13]:
with open("../figi.txt", 'r') as f:
    figis = f.read().split('\n')
len(figis)

83

In [20]:
div_data.to_csv('div_data.csv', index = False)

In [17]:
div_data = await get_dividends_async(figis, pd.Timestamp('2015-01-01'), pd.Timestamp('2024-10-31'), token=TOKEN)

In [21]:
%%time
data = get_prices_sync(figis, pd.Timestamp('2015-01-01'), pd.Timestamp('2024-10-31'), token=TOKEN)

CPU times: user 25.6 s, sys: 743 ms, total: 26.3 s
Wall time: 1min 40s


In [24]:
data.to_parquet("../data/tinvest_ohlc.parquet", index = False)

## assets

In [63]:
# asset_df = _dataclasses_to_dict(asset_resp.asset)
# for key in asset_df:
#     print(key, type(asset_df[key]))

In [42]:
asset_df.columns

Index(['uid', 'type', 'name', 'name_brief', 'description', 'deleted_at',
       'required_tests', 'currency', 'gos_reg_code', 'cfi', 'code_nsd',
       'status', 'updated_at', 'br_code', 'br_code_name', 'instruments',
       'security.isin', 'security.type', 'security.instrument_kind',
       'security.share.type', 'security.share.issue_size',
       'security.share.nominal', 'security.share.nominal_currency',
       'security.share.primary_index', 'security.share.dividend_rate',
       'security.share.preferred_share_type', 'security.share.ipo_date',
       'security.share.registry_date', 'security.share.div_yield_flag',
       'security.share.issue_kind', 'security.share.placement_date',
       'security.share.repres_isin', 'security.share.issue_size_plan',
       'security.share.total_float', 'security.bond', 'security.sp',
       'security.etf', 'security.clearing_certificate', 'brand.uid',
       'brand.name', 'brand.description', 'brand.info', 'brand.company',
       'brand.secto

In [40]:
enum_cols = ['trading_status', 'share_type', 'real_exchange', 'instrument_exchange']
share_cols = [
    'figi', 'ticker', 'class_code', 'isin', 'lot', 'currency','short_enabled_flag', 'name', 'exchange', 
    'ipo_date', 'issue_size', 'country_of_risk', 'country_of_risk_name', 'sector', 'issue_size_plan',
    'nominal', 'trading_status', 'otc_flag', 'buy_available_flag', 'sell_available_flag', 'div_yield_flag', 
    'share_type', 'min_price_increment', 'api_trade_available_flag', 'uid', 'real_exchange', 'position_uid', 
    'asset_uid', 'instrument_exchange', 'for_iis_flag', 'for_qual_investor_flag', 'weekend_flag', 'blocked_tca_flag', 
    'liquidity_flag', 'first_1min_candle_date', 'first_1day_candle_date']

In [10]:
asset_arr.assets[0].instruments

[AssetInstrument(uid='44a1e3e8-b813-46dd-b1f8-d0e9d91c06d2', figi='BBG000BVPG41', instrument_type='share', ticker='RTO', class_code='SPBXM', links=[], instrument_kind=<InstrumentType.INSTRUMENT_TYPE_SHARE: 2>, position_uid='72161247-8e86-4df5-a911-d06b92620d82')]

In [32]:
res = []
for asset in _dataclasses_to_dict(assets_resp.assets):
    for instrument in asset['instruments']:
        tmp = instrument
        for col in asset:
            if col != 'instruments':
                tmp[f"asset_{col}"] = asset[col]
        res.append(tmp)
assets_df = pd.json_normalize(res)
assets_df.shape

(3424, 11)

In [34]:
assets_df.to_csv("../data/assets_info.csv", index = False)

In [35]:
assets_df.query("class_code == 'TQBR'")

,uid,figi,instrument_type,ticker,class_code,links,instrument_kind,position_uid,asset_uid,asset_type,asset_name
1,2dfbc1fd-b92a-436e-b011-928c79e805f2,BBG004S68473,share,IRAO,TQBR,[],2,427f9bcc-2cab-4561-bf94-942d4261fbb7,efee0066-e7ca-4f56-a671-b22135b0e042,4,"Акции обыкновенные ПАО ""Интер РАО"""
21,ba9b6eb4-614c-4be8-bdba-dd86cdfece64,BBG000Q7GG57,share,TGKB,TQBR,[],2,80a22970-659c-45a5-b0ed-1f842ecd082c,55bb33e5-da31-46bb-8693-d3e9789d9302,4,"Акции обыкновенные ПАО ""ТГК-2"""
24,4563f7a1-8245-4caf-aba5-ac49827ba775,BBG000SK7JS5,share,LNZL,TQBR,[],2,97ba343f-cf64-4460-9ef4-3ccd8673eac3,c6afcf42-d81f-4c8a-86de-aa302b20f171,4,"Акции обыкновенные ПАО ""Лензолото"""
36,7bedd86b-478d-4742-a28c-29d27f8dbc7d,BBG0100R9963,share,SGZH,TQBR,[],2,5fe05020-a682-4909-b9f6-93e91413fa93,9f1ce2d9-a6e9-421f-8451-14207ba5830a,4,"Акции обыкновенные ПАО ""Сегежа Групп"""
96,c1a3c440-f51c-4a75-a400-42a2a74f5f2b,BBG004S68FR6,share,MTLRP,TQBR,[],2,e1eecf7e-95f1-42f7-9a64-a4e43eae7acc,cb1c065d-9d85-4aeb-9e31-41fafb7db8e1,4,"Акции привилегированные ПАО ""Мечел"""
...,...,...,...,...,...,...,...,...,...,...,...
3334,bc21fb4f-8838-4355-8697-fb0d8fc809c8,BBG000GQSVC2,share,NKNCP,TQBR,[],2,7b7201cb-fe73-49a8-95d6-bd5edf4b849c,9e876ded-e160-4b33-b8f2-c1d5b08eaef7,4,"Акции привилегированные ПАО ""Нижнекамскнефтехим"""
3351,3879e073-f27a-4e2f-9a97-97475b25194d,BBG0029SG1C1,share,KZOSP,TQBR,[],2,99a43dec-f6fe-419e-aeaf-5ad4a70d160d,37f51c1f-6a6e-49be-aba1-78b94ae35fa5,4,"Акции привилегированные ПАО ""Казаньоргсинтез"""
3369,a5776620-1e2f-47ea-bbd6-06d8e4a236d8,BBG004S686N0,share,BANEP,TQBR,[],2,aeaa8f4a-5149-49ed-8b9d-816b739bc882,7280bd17-42d6-41b5-b76c-7e231931483f,4,"Акции привилегированные типа А ПАО АНК ""Башнефть"""
3385,eb4ba863-e85f-4f80-8c29-f2627938ee58,BBG004S68598,share,MTLR,TQBR,[],2,ad00cb52-0cf5-48b3-81d1-b842493a35ae,b9f09cfa-1099-4691-a981-8b1a4c0c72f3,4,"Акции обыкновенные ПАО ""Мечел"""


## shares

In [19]:
# res = []
# for share in shares_arr.instruments:
#     tmp = pd.json_normalize(
#         _dataclasses_to_dict(share)
#     )
#     res.append(tmp)
shares_df = pd.concat(res)

In [42]:
shares_df['first_1min_candle_date'].describe()

count                            12072
mean     1981-03-07 15:21:08.782306176
min                1970-01-01 00:00:00
25%                1970-01-01 00:00:00
50%                1970-01-01 00:00:00
75%                1970-01-01 00:00:00
max                2024-10-22 00:00:00
Name: first_1min_candle_date, dtype: object

In [22]:
shares_df.to_csv("../data/shares_info.csv", index = False)

In [50]:
shares_df['lot'].value_counts()

lot
1          11638
10           217
100          131
1000          50
10000         25
100000         9
1000000        2
Name: count, dtype: int64

In [44]:
shares_df

(12111, 45)

In [47]:
short_list = (
    shares_df
    .query('not for_qual_investor_flag and short_enabled_flag and api_trade_available_flag and real_exchange == 1')
    [['figi', 'ticker', 'class_code', 'isin', 'lot', 'currency', 'name', 'exchange', 'asset_uid']]
)

In [48]:
short_list.iloc[0]

figi                                  TCS00A106YF0
ticker                                        VKCO
class_code                                    TQBR
isin                                  RU000A106YF0
lot                                              1
currency                                       rub
name                                            ВК
exchange                       MOEX_DEALER_WEEKEND
asset_uid     f0a6fe09-3981-4501-a8f5-91821935e4d4
Name: 582, dtype: object

In [45]:
assets_df[assets_df['uid'].isin(short_list['uid'])]

,uid,figi,instrument_type,ticker,class_code,links,instrument_kind,position_uid,asset_uid,asset_type,asset_name
1,2dfbc1fd-b92a-436e-b011-928c79e805f2,BBG004S68473,share,IRAO,TQBR,[],2,427f9bcc-2cab-4561-bf94-942d4261fbb7,efee0066-e7ca-4f56-a671-b22135b0e042,4,"Акции обыкновенные ПАО ""Интер РАО"""
36,7bedd86b-478d-4742-a28c-29d27f8dbc7d,BBG0100R9963,share,SGZH,TQBR,[],2,5fe05020-a682-4909-b9f6-93e91413fa93,9f1ce2d9-a6e9-421f-8451-14207ba5830a,4,"Акции обыкновенные ПАО ""Сегежа Групп"""
96,c1a3c440-f51c-4a75-a400-42a2a74f5f2b,BBG004S68FR6,share,MTLRP,TQBR,[],2,e1eecf7e-95f1-42f7-9a64-a4e43eae7acc,cb1c065d-9d85-4aeb-9e31-41fafb7db8e1,4,"Акции привилегированные ПАО ""Мечел"""
122,46ae47ee-f409-4776-bf20-43a040b9e7fb,BBG007N0Z367,share,AGRO,TQBR,[],2,f8a4d4f3-92a8-4f09-b77b-127d588ddf04,65479988-3904-4f1b-a15f-8f3ad99b0234,4,Глобальная депозитарная расписка на обыкновенн...
235,e2bd2eba-75de-4127-b39c-2f2dbe3866c3,BBG000RMWQD4,share,ENPG,TQBR,[],2,62dae90c-238b-433d-ae4e-47c72b324bc7,9b9cb779-d2c7-4e49-a6c3-5be1e08d9439,4,"Акции обыкновенные МКПАО ""ЭН+ ГРУП"""
...,...,...,...,...,...,...,...,...,...,...,...
3167,1ffe1bff-d7b7-4b04-b482-34dc9cc0a4ba,BBG0047315D0,share,SNGS,TQBR,[],2,87a7f6b3-f42d-423e-b213-1a013ea30ef3,56195dde-fd1a-4026-b8e0-42dbda342c7c,4,"Акции обыкновенные ПАО ""Сургутнефтегаз"""
3267,435107a9-a262-4a31-8a9b-2ee6f81c1184,BBG004S68696,share,RASP,TQBR,[],2,43455146-4592-45e0-b277-864173a529ec,df9a9da8-0a95-41ca-9d03-82de3fcf872c,4,"Акции обыкновенные ПАО ""Распадская"""
3369,a5776620-1e2f-47ea-bbd6-06d8e4a236d8,BBG004S686N0,share,BANEP,TQBR,[],2,aeaa8f4a-5149-49ed-8b9d-816b739bc882,7280bd17-42d6-41b5-b76c-7e231931483f,4,"Акции привилегированные типа А ПАО АНК ""Башнефть"""
3385,eb4ba863-e85f-4f80-8c29-f2627938ee58,BBG004S68598,share,MTLR,TQBR,[],2,ad00cb52-0cf5-48b3-81d1-b842493a35ae,b9f09cfa-1099-4691-a981-8b1a4c0c72f3,4,"Акции обыкновенные ПАО ""Мечел"""


In [61]:
short_list['figi'].to_csv('../figi.txt', index = False, header=False)

## moex

In [4]:
from src.moex_api import get_market_candles
import httpx
import asyncio

In [13]:
# %%time
start = pd.Timestamp('2016-01-01')
end = pd.Timestamp('2022-01-01')
secids = None
async with httpx.AsyncClient(timeout=httpx.Timeout(timeout=None)) as session:
    async def process_candles(secid):
        ohlc_tmp = await get_market_candles(session, security = secid, start = start, end = end)
        ohlc_tmp = pd.DataFrame(ohlc_tmp)
        ohlc_tmp['secid'] = secid
        return ohlc_tmp
    request_url = "https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json"
    moex_info_resp = (await session.get(request_url)).raise_for_status().json()
    moex_info_df = pd.DataFrame(moex_info_resp["securities"]['data'], 
                                columns = moex_info_resp["securities"]['columns'])
    moex_info_df.columns = moex_info_df.columns.str.lower()
    if secids is None:
        secids = moex_info_df['secid']
    tasks = [process_candles(secid) for secid in secids]
    ohlc_data = await asyncio.gather(*tasks)
moex_candles = pd.concat(ohlc_data, axis = 0)

In [15]:
moex_candles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 277892 entries, 0 to 1437
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    277892 non-null  float64
 1   close   277892 non-null  float64
 2   high    277892 non-null  float64
 3   low     277892 non-null  float64
 4   value   277892 non-null  float64
 5   volume  277892 non-null  float64
 6   begin   277892 non-null  object 
 7   end     277892 non-null  object 
 8   secid   277892 non-null  object 
dtypes: float64(6), object(3)
memory usage: 21.2+ MB


In [11]:
# async with httpx.AsyncClient() as session:
#     ohlc_data = []
#     async for secid in moex_info_df['secid']:
#         print(secid)
#         ohlc_tmp = await get_market_candles(session, security = secid, start = start, end = end)
#         ohlc_tmp = pd.DataFrame(ohlc_tmp)
#         ohlc_tmp['secid'] = secid
#         ohlc_data.append(ohlc_tmp)
moex_candles = pd.concat(ohlc_data, axis = 0)

In [17]:
pd.Timestamp.today().normalize()

Timestamp('2024-11-08 00:00:00')

In [15]:
# tmp = pd.read_parquet("../data/tinvest_ohlc.parquet")
tmp = pd.read_csv("../data/moex_tqbr_ohlc.csv")
tmp.columns

Index(['open', 'close', 'high', 'low', 'value', 'volume', 'begin', 'end',
       'secid'],
      dtype='object')